# User Latent Dirichlet Allocation

In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import pickle
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('.'))
torch.autograd.set_detect_anomaly(True)

In [2]:
%load_ext autoreload
%autoreload 2

from lib.models import InductiveLDA, NeuralLDA, NeuralLLNA
from lib import utils as utils

Import Data

In [3]:
class Corpus:
    def __init__(self, datadir):
        filenames = ['train.txt.npy', 'test.txt.npy']
        self.datapaths = [os.path.join(datadir, x) for x in filenames]
        with open(os.path.join(datadir, 'vocab.pkl'), 'rb') as f:
            self.vocab = pickle.load(f)
        self.train, self.test = [
            Data(dp, len(self.vocab)) for dp in self.datapaths]


class Data:
    def __init__(self, datapath, vocab_size):
        data = np.load(datapath, allow_pickle=True, encoding='bytes')
        self.data = np.array([np.bincount(x.astype('int'), minlength=vocab_size) for x in data if np.sum(x)>0])
        self.documents = data
        
    @property
    def size(self):
        return len(self.data)
    
    def get_batch(self, batch_size, start_id=None):
        if start_id is None:
            batch_idx = np.random.choice(np.arange(self.size), batch_size)
        else:
            batch_idx = np.arange(start_id, start_id + batch_size)
        batch_data = self.data[batch_idx]
        data_tensor = torch.from_numpy(batch_data).float()
        return data_tensor

In [4]:
corpus = Corpus("../data/20news")

In [5]:
RANDOM_SEED = 2112
np.random.seed(RANDOM_SEED)

In [6]:
X_document = corpus.train.data

In [7]:
num_docs, vocab_size = X_document.shape
print(f"Number of documents: {num_docs}, vocab size: {vocab_size}")

Number of documents: 11258, vocab size: 1995


In [8]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X): self.X = X
    def __len__(self): return len(self.X)
    def __getitem__(self, index): return self.X[index]

In [9]:
dataset = Dataset(torch.tensor(X_document).to(torch.float32))

In [25]:
num_topics = 50
prodlda = True
conv = False
num_layers = 0 # num hidden layers in the NN block. There are at least (1+num_heads) hidden layers in the encoder and decoder. num_layers comes on top of those. Total number of hidden layers is (1+num_layers+num_heads).
num_neurons = 100
dropout = True
dropout_rate = 0.25
batch_normalization = True
prior_param = {'alpha': 1.0}
decoder_temperature = 1.0
encoder_temperature = 1.0

In [26]:
# NeuralModel, prior_param = NeuralLLNA, {"mu": 0.0, "sigma": 1.0}
# NeuralModel = NeuralLDA
NeuralModel = InductiveLDA

In [27]:
model = NeuralModel(input_dim=vocab_size,
                    num_topics=num_topics,
                    prior_param=prior_param.copy(),
                    conv=conv,
                    prodlda=prodlda,
                    decoder_temperature=decoder_temperature,
                    num_hidden_layers=num_layers,
                    num_neurons=num_neurons,
                    dropout=dropout,
                    dropout_rate=dropout_rate,
                    batch_normalization=batch_normalization,
                    )

In [28]:
lr = 5e-3
batch_size = 200
num_epochs = 100
beta = 1.0
learn_prior = True

In [29]:
# Train the model using default partitioning choice 
model.fit(lr=lr,
            dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True),
            epochs = num_epochs,
            beta = beta,
            mc_samples = 1,
            learn_prior = learn_prior,
            tensorboard = False,
            )

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

                                                      
Epoch:   2%|▏         | 2/100 [00:03<01:52,  1.15s/it]             

Iteration: 100 -- ELBO=-5.48e+02 / RLL=-5.42e+02 / KL=6.66e+00


                                                      
Epoch:   4%|▍         | 4/100 [00:05<01:55,  1.20s/it]             

Iteration: 200 -- ELBO=-9.29e+02 / RLL=-9.21e+02 / KL=7.70e+00


...

Iteration: 5500 -- ELBO=-7.28e+02 / RLL=-7.17e+02 / KL=1.12e+01


                                                       
Epoch:  99%|█████████▉| 99/100 [02:04<00:01,  1.26s/it]            

Iteration: 5600 -- ELBO=-6.87e+02 / RLL=-6.76e+02 / KL=1.10e+01


                                                        
Iteration in Epoch: 100%|██████████| 57/57 [00:01<00:00, 45.15it/s]

Iteration: 5700 -- ELBO=-4.69e+02 / RLL=-4.60e+02 / KL=9.87e+00


In [30]:
model.get_prior_params()

tensor([7.8248e-01, 8.5697e-04, 2.8882e-03, 8.3738e-01, 7.6415e-04, 9.1559e-01,
        7.2740e-01, 5.1719e-01, 5.8330e-04, 1.0756e-03, 1.3546e-03, 4.2201e-01,
        8.2058e-04, 9.3865e-01, 6.5549e-01, 5.0389e-03, 8.6805e-03, 5.8849e-04,
        9.9697e-04, 7.9068e-04, 7.6066e-04, 1.1435e-03, 6.6645e-04, 1.0982e-03,
        2.4720e-01, 1.2482e-03, 6.9792e-03, 8.4468e-04, 1.1672e-03, 7.0699e-04,
        9.9519e-04, 5.5826e-04, 6.2155e-01, 1.4910e-02, 8.2116e-04, 2.3724e-02,
        6.4800e-01, 1.0430e-02, 5.6161e-04, 9.4454e-03, 4.5106e-02, 7.0446e-01,
        1.4588e-03, 8.6612e-04, 8.2152e-04, 7.6196e-03, 7.3882e-01, 4.3006e-03,
        7.4542e-04, 6.6248e-01], grad_fn=<SoftplusBackward0>)

In [31]:
model.decoder.get_beta().detach()

tensor([[3.2186e-08, 2.1772e-03, 1.2023e-03,  ..., 9.7794e-07, 3.6909e-07,
         1.1403e-08],
        [2.8716e-06, 2.5772e-02, 1.9077e-03,  ..., 1.3827e-05, 3.5287e-06,
         3.5271e-06],
        [1.8509e-05, 1.5041e-03, 6.6409e-04,  ..., 7.3777e-06, 6.2625e-06,
         2.5380e-06],
        ...,
        [1.8513e-06, 7.4842e-03, 4.8255e-03,  ..., 1.2720e-05, 3.0170e-06,
         2.9274e-06],
        [1.6940e-06, 1.1498e-02, 2.6513e-03,  ..., 3.4772e-05, 1.3194e-05,
         3.1730e-06],
        [1.0509e-08, 2.7682e-03, 3.2106e-03,  ..., 3.1044e-08, 2.5065e-07,
         1.9819e-08]])

In [32]:
model.eval()

InductiveLDA(
  (encoder): DirichletNN(
    (parameterizer): ParameterizerNN(
      (block_dict): ModuleDict(
        (input): NNBlock(
          (input_layer): Sequential(
            (0): Linear(in_features=1995, out_features=100, bias=True)
            (1): Softplus(beta=1, threshold=20)
          )
          (middle_layers): ModuleList()
          (output_layer): Sequential(
            (0): Linear(in_features=100, out_features=100, bias=True)
            (1): Softplus(beta=1, threshold=20)
          )
        )
        (alpha): NNBlock(
          (input_layer): Sequential(
            (0): Linear(in_features=100, out_features=100, bias=True)
            (1): Softplus(beta=1, threshold=20)
            (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): Dropout(p=0.25, inplace=False)
          )
          (middle_layers): ModuleList()
          (output_layer): Sequential(
            (0): Linear(in_features=100, out_features=50, bia

In [33]:
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
import gensim.corpora as corpora

In [34]:
## turn the encoding in corpus.train.documents into a list of list of words using the vocab
id2word = {v: k for k, v in corpus.vocab.items()}
texts = [[id2word[i] for i in doc] for doc in corpus.train.documents]

In [35]:
Beta = model.decoder.get_beta().detach().numpy()
# Beta = model.decoder.beta_unnorm.detach().numpy()

In [36]:
result = {}
result["topic-word-matrix"] = Beta

top_k = 10

if top_k > 0:
    topics_output = []
    for topic in result["topic-word-matrix"]:
        top_k_words = list(reversed([id2word[i] for i in np.argsort(topic)[-top_k:]]))
        topics_output.append(top_k_words)
    result["topics"] = topics_output

In [37]:
result["topics"]

[['ide',
  'scsi',
  'bus',
  'isa',
  'scsus',
  'controller',
  'simm',
  'drive',
  'team',
  'mhz'],
 ['article',
  'mean',
  'write',
  'good',
  'think',
  'anyone',
  'player',
  'way',
  'claim',
  'please'],
 ['dos',
  'world',
  'cd',
  'war',
  'ii',
  'family',
  'great',
  'money',
  'library',
  'business'],
 ['bike',
  'ride',
  'game',
  'fire',
  'dog',
  'helmet',
  'cop',
  'dod',
  'armenians',
  'hit'],
 ['anyone',
  'fax',
  'article',
  'think',
  'please',
  'know',
  'gun',
  'thing',
  'way',
  'car'],
 ['drive',
  'car',
  'connector',
  'monitor',
  'voltage',
  'cable',
  'pin',
  'printer',
  'floppy',
  'sale'],
 ['god',
  'eternal',
  'ride',
  'drug',
  'bike',
  'christianity',
  'life',
  'faith',
  'hell',
  'christian'],
 ['armenian',
  'turkish',
  'armenians',
  'israel',
  'armenia',
  'turks',
  'israeli',
  'genocide',
  'village',
  'lebanese'],
 ['write',
  'god',
  'israeli',
  'human',
  'life',
  'faith',
  'want',
  'tell',
  'article',
 

In [38]:
# Initialize metric
npmi = CoherenceModel(
    topics=result["topics"],
    texts=texts,
    corpus=corpus.train.data,
    dictionary=Dictionary(texts),
    coherence="c_npmi",
    topn=top_k)

In [39]:
npmi.get_coherence()

0.06962826290808065